# 1. Transformación datos bronze

In [ ]:
import pandas as pd
import json
from pathlib import Path
import base64
import requests

# Procesar todos los archivos .csv de la carpeta de entrada
for archivo_csv in archivos:
    try:
        url_csv = f"{raw_base_url}/{archivo_csv}" # Changed 'archivo' to 'archivo_csv' to use the correct filename in the loop


        # Cargar el CSV
        df_raw = pd.read_csv(url_csv, sep=';', parse_dates=['time'])

        # Si tiene columna 'attributes', expandirla
        if 'attributes' in df_raw.columns:
            df_raw['attributes'] = df_raw['attributes'].apply(json.loads)
            atributos_expandidos = pd.json_normalize(df_raw['attributes'])
            df_limpio = pd.concat([df_raw.drop(columns=['attributes']), atributos_expandidos], axis=1)
        else:
            df_limpio = df_raw


        # Nombre del archivo limpio
        nombre_limpio = f"{Path(archivo_csv).stem}_desglosados.csv"

        # Ruta del contenido en el repo donde subirás los archivos limpios
        upload_path = "data/data_silver/csv_colab/Files_desglosados"

        guardar_ficheros(owner, repo, upload_path, nombre_limpio, df_limpio)

    except Exception as e:
        print(f"⚠️ Error al procesar {archivo_csv}: {e}")


✅ CSV_datos_Sol_desglosados.csv subido correctamente al repo.
⚠️ Error al procesar History_2024-09-25_100-2025-03-26_1300-2B2.csv: Missing column provided to 'parse_dates': 'time'
✅ entidad_sensor_mislata_desglosados.csv subido correctamente al repo.
✅ entidad_sensor_sensor_desglosados.csv subido correctamente al repo.
✅ entidad_weather_forecast_desglosados.csv subido correctamente al repo.


In [ ]:
import pandas as pd

path_read = 'data/data_bronze/History_2024-09-25_100-2025-03-26_1300-2B2.csv'
df_historico = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

# Primero, unimos la columna 'fecha' con la columna 'tiempo'
df_historico['time'] = pd.to_datetime(df_historico['date'].astype(str) + ' ' + df_historico['time'].astype(str))

# Formateamos la columna 'time' para que sea como 'YYYY-MM-DD HH:MM:SS'
df_historico['time'] = df_historico['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Eliminamos la columna 'humeda'
df_historico.drop(columns=['date', 'humidity %RH'], inplace=True)

# Guardar el nuevo archivo
upload_path = "data/data_silver"
guardar_ficheros(owner, repo, upload_path, 'Historico_temperaturas.csv', df_historico)

<ipython-input-7-a6a4ea0b9431>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_historico = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])
<ipython-input-7-a6a4ea0b9431>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_historico['time'] = pd.to_datetime(df_historico['date'].astype(str) + ' ' + df_historico['time'].astype(str))


✅ Historico_temperaturas.csv subido correctamente al repo.


# Transformación de Plata

In [ ]:
# Cargar el archivo de sensores limpios
path_read = 'data/data_silver/csv_colab/Files_desglosados/entidad_sensor_sensor_desglosados.csv'
df_sensores = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

# Filtrar 'unavailable' (manteniendo numéricos, 'on' y 'off')
df_sensores_filtrado = df_sensores[df_sensores['state'] != 'unavailable'].copy()

# Identificar las filas que NO son 'on' ni 'off'
mask_numerico = ~df_sensores_filtrado['state'].isin(['on', 'off'])

# Convertir a numérico solo donde corresponde
df_sensores_filtrado.loc[mask_numerico, 'state'] = pd.to_numeric(df_sensores_filtrado.loc[mask_numerico, 'state'], errors='coerce')

# Convertir unidades inHg -> hPa
cond_inhg = (df_sensores_filtrado['unit_of_measurement'] == 'inHg') & mask_numerico
df_sensores_filtrado.loc[cond_inhg, 'state'] = df_sensores_filtrado.loc[cond_inhg, 'state'].astype(float) * 33.8639
df_sensores_filtrado.loc[cond_inhg, 'unit_of_measurement'] = 'hPa'

# Redondear presión a un decimal
cond_presion = (df_sensores_filtrado['unit_of_measurement'] == 'hPa') & mask_numerico
df_sensores_filtrado.loc[cond_presion, 'state'] = df_sensores_filtrado.loc[cond_presion, 'state'].astype(float).round(1)

# Formatear la columna 'time'
df_sensores_filtrado['time'] = pd.to_datetime(df_sensores_filtrado['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_sensores_filtrado['time'] = pd.to_datetime(df_sensores_filtrado['time'])  # volver a datetime para agrupar

# Guardar el archivo
upload_path = "data/data_silver/csv_colab"
guardar_ficheros(owner, repo, upload_path, 'Sensores_limpio.csv', df_sensores_filtrado)

print(f'Archivo guardado correctamente en: {upload_path}')

✅ Sensores_limpio.csv subido correctamente al repo.
Archivo guardado correctamente en: data/data_silver/csv_colab


In [ ]:
# Cargar el archivo de sensores limpios
path_read = 'data/data_silver/csv_colab/Sensores_limpio.csv'
df_sensores = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

# Agrupar por entity_id y por intervalos de 60 minutos
df_agrupado = (
    df_sensores_filtrado[mask_numerico].dropna(subset=['state'])
    .groupby([
        'entity_id',
        pd.Grouper(key='time', freq='60T')
    ])
    .agg(valor_medio=('state', 'mean'))
    .reset_index()
)

# Guardar el archivo
upload_path = "data/data_silver/csv_colab"
guardar_ficheros(owner, repo, upload_path, 'Sensores_agrupado.csv', df_agrupado)

<ipython-input-9-1dde9178359b>:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  pd.Grouper(key='time', freq='60T')


✅ Sensores_agrupado.csv subido correctamente al repo.


In [ ]:
# Cargar el archivo filtrado
path_read = 'data/data_silver/csv_colab/Sensores_agrupado.csv'
df = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

# Asegurar que 'valor_medio' sea numérico
df['valor_medio'] = pd.to_numeric(df['valor_medio'], errors='coerce')

# Filtrar valores atípicos: eliminar valores mayores a 100 para sensores de temperatura (por seguridad)
import re
pattern_temp = re.compile(r'sensor\.sensor_temperatura_\d+_temperature')
es_sensor_temp = df['entity_id'].apply(lambda x: bool(pattern_temp.fullmatch(x)))
df = df[~((es_sensor_temp) & (df['valor_medio'] > 100))]

# Redondear 'valor_medio' a 2 decimales
df['valor_medio'] = df['valor_medio'].round(2)

# Guardar el archivo corregido final
upload_path = "data/data_silver/csv_colab"
guardar_ficheros(owner, repo, upload_path, 'Sensores_agrupado.csv', df)

✅ Sensores_agrupado.csv subido correctamente al repo.


In [ ]:
import pandas as pd

path_read = 'data/data_silver/csv_colab/Sensores_agrupado.csv'
df = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

df_sin_bateria = df[~df['entity_id'].str.contains('_battery', na=False)]

# Eliminamos filas que contienen un sensor nuevo que solo hace un registro 3 veces
df_sin_bateria = df_sin_bateria.drop(df[df['entity_id'].isin(['sensor.sensor_temperatura_3_nuevo_4_humidity','sensor.sensor_temperatura_3_nuevo_4_pressure','sensor.sensor_temperatura_3_nuevo_4_temperature'])].index)


sensores_ventanas_y_puerta = df_sin_bateria[
    (df_sin_bateria['entity_id'].str.contains('sensor.sensor_ventana_', na=False)) |
    (df_sin_bateria['entity_id'] == 'sensor.sensor_puerta_1_device_temperature')
]

sensores_temperatura = df_sin_bateria[df_sin_bateria['entity_id'].str.contains('sensor.sensor_temperatura_', na=False)]

upload_path = "data/data_silver/Agrupation"

guardar_ficheros(owner, repo, upload_path, 'Sensores_ventanas_y_puerta_agrupados.csv', sensores_ventanas_y_puerta)
guardar_ficheros(owner, repo, upload_path, 'Sensores_temperatura_clase_agrupados.csv', sensores_temperatura)

✅ Sensores_ventanas_y_puerta_agrupados.csv subido correctamente al repo.
✅ Sensores_temperatura_clase_agrupados.csv subido correctamente al repo.


In [ ]:
path_read = 'data/data_silver/csv_colab/Files_desglosados/CSV_datos_Sol_desglosados.csv'
df_sol_sin_agrupar = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

df_sol_sin_agrupar = df_sol_sin_agrupar[["time", "azimuth", "elevation"]]

# Intentar convertir la columna 'time' asegurando compatibilidad con formatos variables
df_sol_sin_agrupar['time'] = pd.to_datetime(df_sol_sin_agrupar['time'], format='ISO8601', errors='coerce')

# Remover la zona horaria si existe
df_sol_sin_agrupar['time'] = df_sol_sin_agrupar['time'].dt.tz_localize(None)

# Convertir las columnas a numérico, ignorando errores
df_sol_sin_agrupar['azimuth'] = pd.to_numeric(df_sol_sin_agrupar['azimuth'], errors='coerce')
df_sol_sin_agrupar['elevation'] = pd.to_numeric(df_sol_sin_agrupar['elevation'], errors='coerce')

# Agrupar por intervalos de 60 minutos y calcular la media de 'azimuth' y 'elevation'
df_sol_agrupado = df_sol_sin_agrupar.groupby(pd.Grouper(key='time', freq='60T')).agg(
    azimuth_mean=('azimuth', 'mean'),
    elevation_mean=('elevation', 'mean')
).reset_index()

# Redondear 'valor_medio' a 2 decimales
df_sol_agrupado['azimuth_mean'] = df_sol_agrupado['azimuth_mean'].round(2)
df_sol_agrupado['elevation_mean'] = df_sol_agrupado['elevation_mean'].round(2)


df_sol_agrupado.to_csv("./datos_sol_agrupados.csv", index=False)
df_sol_agrupado

upload_path = "data/data_silver/Agrupation"
guardar_ficheros(owner, repo, upload_path, 'datos_sol_agrupados.csv', df_sol_agrupado)

<ipython-input-12-cfef7d1ff4f8>:17: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_sol_agrupado = df_sol_sin_agrupar.groupby(pd.Grouper(key='time', freq='60T')).agg(


✅ datos_sol_agrupados.csv subido correctamente al repo.


In [ ]:
import json

path_read = 'data/data_silver/csv_colab/Files_desglosados/entidad_weather_forecast_desglosados.csv'
df_limpio = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

df_silver = df_limpio[df_limpio['entity_id'] != "weather.forecast_casa"]
df_silver = df_silver[df_silver['state'] != 'unavailable']

# Nos quedamos con las columnas que nos interesan
df_silver = df_silver[["time", "temperature", "cloud_coverage"]]
df_silver['time'] = pd.to_datetime(df_silver['time']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Intentar convertir la columna 'time' asegurando compatibilidad con formatos variables
df_silver['time'] = pd.to_datetime(df_silver['time'], format='ISO8601', errors='coerce')

# Remover la zona horaria si existe
df_silver['time'] = df_silver['time'].dt.tz_localize(None)

# Convertir las columnas a numérico, ignorando errores
df_silver['temperature'] = pd.to_numeric(df_silver['temperature'], errors='coerce')
df_silver['cloud_coverage'] = pd.to_numeric(df_silver['cloud_coverage'], errors='coerce')

# Agrupar por intervalos de 30 minutos y calcular la media de 'temperature' y 'cloud_coverage'
df_forecast_agrupado = df_silver.groupby(pd.Grouper(key='time', freq='60T')).agg(
    temperature_mean=('temperature', 'mean'),
    cloud_coverage_mean=('cloud_coverage', 'mean')
).reset_index()

# Redondear 'valor_medio' a 2 decimales
df_forecast_agrupado['temperature_mean'] = df_forecast_agrupado['temperature_mean'].round(2)
df_forecast_agrupado['cloud_coverage_mean'] = df_forecast_agrupado['cloud_coverage_mean'].round(2)


upload_path = "data/data_silver/Agrupation"
guardar_ficheros(owner, repo, upload_path, 'datos_pronostico_agrupado.csv', df_forecast_agrupado)

<ipython-input-13-936d37f41edf>:24: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_forecast_agrupado = df_silver.groupby(pd.Grouper(key='time', freq='60T')).agg(


✅ datos_pronostico_agrupado.csv subido correctamente al repo.


In [ ]:
import pandas as pd

# Lista de columnas que quieres en la salida
desired_columns = [
    'sensor.sensor_temperatura_1_humidity',
    'sensor.sensor_temperatura_1_pressure',
    'sensor.sensor_temperatura_1_temperature',
    'sensor.sensor_temperatura_2_humidity',
    'sensor.sensor_temperatura_2_pressure',
    'sensor.sensor_temperatura_2_temperature',
    'sensor.sensor_temperatura_3_humidity',
    'sensor.sensor_temperatura_3_pressure',
    'sensor.sensor_temperatura_3_temperature',
    'sensor.sensor_temperatura_4_humidity',
    'sensor.sensor_temperatura_4_pressure',
    'sensor.sensor_temperatura_4_temperature'
]

# 1. Leer el CSV
path_read = 'data/data_silver/Agrupation/Sensores_temperatura_clase_agrupados.csv'
df = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

# 2. Asegurar que 'time' es índice
df.set_index("time", inplace=True)

# 3. Pivotar la tabla para tener cada entity_id como columna
df_pivot = df.pivot(columns="entity_id", values="valor_medio")

# 4. Crear un rango de fechas cada 60 minutos (media hora) desde la min a la max
all_times = pd.date_range(start=df_pivot.index.min(), end=df_pivot.index.max(), freq="60T")

# 5. Reindexar y rellenar valores ausentes con el último valor disponible
df_pivot = df_pivot.reindex(all_times)
df_pivot = df_pivot.ffill()

# 6. Reindexar para quedarnos sólo con las columnas deseadas
df_pivot = df_pivot.reindex(columns=desired_columns)

# Opcional: restaurar el índice como columna si prefieres 'time' en lugar de índice
df_pivot.reset_index(inplace=True)
df_pivot.rename(columns={"index": "time"}, inplace=True)


upload_path = "data/data_silver/Agrupation_Final"
guardar_ficheros(owner, repo, upload_path, 'Sensores_Principales_Pivotados.csv', df_pivot)

<ipython-input-14-5aee670514d3>:30: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  all_times = pd.date_range(start=df_pivot.index.min(), end=df_pivot.index.max(), freq="60T")


✅ Sensores_Principales_Pivotados.csv subido correctamente al repo.


In [ ]:
import pandas as pd

'''
Nos hace falta dado el df_contact tenemos que juntar la ultima tabla creada en gold, para que si la puerta se ha abierto en el plazo de los 60 minutos.
Se sumen los segundos y se cree una nueva columna con el tiempo total que ha estado abierta cada ventana/puerta.
'''

path_read = 'data/data_silver/csv_colab/Sensores_limpio.csv'
df = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

df_contact = df[df['entity_id'].str.contains('contact', na=False)]

# Aseguramos que 'time' es datetime y ordenamos por sensor y tiempo
df_contact['time'] = pd.to_datetime(df_contact['time'])
df_contact = df_contact.sort_values(['friendly_name', 'time'])

# Convertimos 'state' a valores binarios: 1 para 'on' y 0 para 'off'
df_contact['state'] = df_contact['state'].map({'on': 1, 'off': 0})

# Lista para ir acumulando los registros distribuidos por intervalo de 30 minutos
records = []

# Procesamos cada sensor por separado
for sensor, group in df_contact.groupby('friendly_name'):
    group = group.sort_values('time').reset_index(drop=True)

    # Recorremos cada fila. Solo nos interesa cuando el sensor está en "on"
    for i, row in group.iterrows():
        if row['state'] == 1:
            start_time = row['time']
            # Se toma la siguiente fila para definir el final del período "on"
            if i < len(group) - 1:
                end_time = group.loc[i + 1, 'time']
            else:
                # Si no hay siguiente registro, se ignora este período
                continue

            # Si por alguna razón end_time es anterior a start_time, se omite
            if end_time <= start_time:
                continue

            # Determinamos el inicio del primer bucket de 30 minutos (se trunca a la hora)
            bucket_start = start_time.floor('60T')

            # Iteramos mientras el bucket actual se traslape con el período "on"
            while bucket_start < end_time:
                bucket_end = bucket_start + pd.Timedelta(minutes=60)
                # Calculamos la intersección entre el período "on" y el intervalo actual
                overlap_start = max(start_time, bucket_start)
                overlap_end = min(end_time, bucket_end)
                overlap_seconds = (overlap_end - overlap_start).total_seconds()
                if overlap_seconds > 0:
                    records.append({
                        'friendly_name': sensor,
                        'time': bucket_start,  # Usamos el inicio del bucket como clave
                        'duration': overlap_seconds
                    })
                bucket_start = bucket_end

# Creamos un DataFrame a partir de los registros
df_records = pd.DataFrame(records)

# Agrupamos por sensor y por cada bucket (intervalo de 30 minutos) sumando las duraciones
df_aggregated = df_records.groupby(['friendly_name', 'time']).agg(segundos_abierta=('duration', 'sum')).reset_index()

# Pivotamos para que cada sensor sea una columna, manteniendo 'time' como índice
df_pivot_contacto = df_aggregated.pivot(index='time', columns='friendly_name', values='segundos_abierta').fillna(0).reset_index()

# Hacemos suma de ventanas
columnas_ventanas_arriba = [
    " sensor_ventana_2 Puerta",
    "sensor_ventana_4 Puerta",
    "sensor_ventana_6 Puerta",
    "sensor_ventana_8 Puerta",
    "sensor_ventana_12 Puerta"
]

# Se multiplica por 0.25 poque las ventanas de arriba tienen menos importancia
df_pivot_contacto["suma_ventanas_arriba"] = df_pivot_contacto[columnas_ventanas_arriba].sum(axis=1) * 0.25

columnas_ventanas_abajo = [
    "sensor_ventana_1 Puerta",
    "sensor_ventana_3 Puerta",
    "sensor_ventana_5 Puerta",
    "sensor_ventana_7 Puerta",
    "sensor_ventana_9 Puerta",
    "sensor_ventana_11 Puerta"
]

df_pivot_contacto["suma_ventanas_abajo"] = df_pivot_contacto[columnas_ventanas_arriba].sum(axis=1)


upload_path = "data/data_silver/Agrupation_Final"
guardar_ficheros(owner, repo, upload_path, 'Sensores_Ventanas&Puertas_Pivotados.csv', df_pivot_contacto)

<ipython-input-15-6363c907bb69>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_contact['time'] = pd.to_datetime(df_contact['time'])
<ipython-input-15-6363c907bb69>:43: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  bucket_start = start_time.floor('60T')
<ipython-input-15-6363c907bb69>:43: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  bucket_start = start_time.floor('60T')
<ipython-input-15-6363c907bb69>:43: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  bucket_start = start_time.floor('60T')
<ipython-input-15-6363c907bb69>:43: FutureWarning: 'T' is deprecated and will be remove

✅ Sensores_Ventanas&Puertas_Pivotados.csv subido correctamente al repo.


In [ ]:
# Esta celda de aquí es provisional, he juntado las últimas 3 celdas en una sola porque utilizaban la misma variable 'df_final'.

path_read = 'data/data_silver/Agrupation_Final/Sensores_Principales_Pivotados.csv'
df_pivot = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

path_read = 'data/data_silver/Agrupation_Final/Sensores_Ventanas&Puertas_Pivotados.csv'
df_pivot_contacto = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

# Fusionar df_pivot con el contacto, dejando solo las columnas que nos interesan para reducir dimensionalidad, en base a 'time'

df_final = df_pivot.merge(df_pivot_contacto[["time", "sensor_puerta_1 Puerta", "suma_ventanas_arriba", "suma_ventanas_abajo"]], on='time', how='left')

cols_a_rellenar = ["sensor_puerta_1 Puerta", "suma_ventanas_arriba", "suma_ventanas_abajo"]

# Rellenar los valores NaN con 0, porque realmente son 0
df_final[cols_a_rellenar] = df_final[cols_a_rellenar].fillna(0)

upload_path = "data/data_silver/Agrupation_Final"
guardar_ficheros(owner, repo, upload_path, 'Sensores_Merge.csv', df_final)

#--------------------------------------------------------#

path_read = 'data/data_silver/Agrupation/datos_sol_agrupados.csv'
df_sol_agrupado = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

df_final = df_final.merge(df_sol_agrupado, on='time', how='left')

#--------------------------------------------------------#

path_read = 'data/data_silver/Agrupation/datos_pronostico_agrupado.csv'
df_forecast_agrupado = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

df_final = df_final.merge(df_forecast_agrupado, on='time', how='left')

#--------------------------------------------------------#

path_read = 'data/data_silver/Historico_temperaturas.csv'
df_y_true = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

df_final = df_final.merge(df_y_true[["time", "temperature ℃"]], on='time', how='left')

upload_path = "data/data_silver"
guardar_ficheros(owner, repo, upload_path, 'final_silver.csv', df_final)

✅ Sensores_Merge.csv subido correctamente al repo.
✅ final_silver.csv subido correctamente al repo.


In [ ]:
import pandas as pd
import requests
from datetime import datetime
from time import sleep

# Ruta del archivo original
path_read = 'data/data_silver/final_silver.csv'
df = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

# Asegurar que 'time' está en formato datetime
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Coordenadas de Mislata, España
latitude = 39.475
longitude = -0.412

# Agrupar fechas únicas del dataset
fechas = sorted(df['time'].dt.strftime('%Y-%m-%d').unique())

# Diccionario para almacenar resultados por fecha
datos_api = {}

# Recolectar datos de Open-Meteo para cada fecha
for fecha in fechas:
    url = (
        "https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={latitude}&longitude={longitude}"
        f"&start_date={fecha}&end_date={fecha}"
        "&hourly=temperature_2m,cloudcover"
        "&timezone=Europe%2FMadrid"
    )
    print(f"Consultando datos para {fecha}...")
    response = requests.get(url)
    if response.status_code == 200:
        datos_api[fecha] = response.json()
    else:
        print(f"⚠ Error al consultar datos de {fecha}")
    sleep(1.2)  # Para evitar bloquear la API

# Reemplazar columnas temperature_mean y cloud_coverage_mean
n_reemplazadas = 0
for idx, row in df.iterrows():
    dt = row['time']
    fecha = dt.strftime('%Y-%m-%d')
    hora = dt.strftime('%H:00')
    try:
        datos_dia = datos_api[fecha]
        idx_hora = datos_dia['hourly']['time'].index(f"{fecha}T{hora}")
        temp = datos_dia['hourly']['temperature_2m'][idx_hora]
        cloud = datos_dia['hourly']['cloudcover'][idx_hora]
        df.at[idx, 'temperature_mean'] = temp
        df.at[idx, 'cloud_coverage_mean'] = cloud
        n_reemplazadas += 1
    except Exception as e:
        print(f"No se pudo reemplazar fila {idx} ({fecha} {hora}): {e}")

# Guardar el nuevo archivo
upload_path = "data/data_silver"
guardar_ficheros(owner, repo, upload_path, 'final_silver.csv', df)
df

Consultando datos para 2024-12-10...
Consultando datos para 2024-12-11...
Consultando datos para 2024-12-12...
Consultando datos para 2024-12-13...
Consultando datos para 2024-12-14...
Consultando datos para 2024-12-15...
Consultando datos para 2024-12-16...
Consultando datos para 2024-12-17...
Consultando datos para 2024-12-18...
Consultando datos para 2024-12-19...
Consultando datos para 2024-12-20...
Consultando datos para 2024-12-21...
Consultando datos para 2024-12-22...
Consultando datos para 2024-12-23...
Consultando datos para 2024-12-24...
Consultando datos para 2024-12-25...
Consultando datos para 2024-12-26...
Consultando datos para 2024-12-27...
Consultando datos para 2024-12-28...
Consultando datos para 2024-12-29...
Consultando datos para 2024-12-30...
Consultando datos para 2024-12-31...
Consultando datos para 2025-01-01...
Consultando datos para 2025-01-02...
Consultando datos para 2025-01-03...
Consultando datos para 2025-01-04...
Consultando datos para 2025-01-05...
C

,time,sensor.sensor_temperatura_1_humidity,sensor.sensor_temperatura_1_pressure,sensor.sensor_temperatura_1_temperature,sensor.sensor_temperatura_2_humidity,sensor.sensor_temperatura_2_pressure,sensor.sensor_temperatura_2_temperature,sensor.sensor_temperatura_3_humidity,sensor.sensor_temperatura_3_pressure,sensor.sensor_temperatura_3_temperature,...,sensor.sensor_temperatura_4_pressure,sensor.sensor_temperatura_4_temperature,sensor_puerta_1 Puerta,suma_ventanas_arriba,suma_ventanas_abajo,azimuth_mean,elevation_mean,temperature_mean,cloud_coverage_mean,temperature ℃
0,2024-12-10 18:00:00,46.443333,1013.5,21.150000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,258.46,-23.01,12.7,99.0,21.9
1,2024-12-10 19:00:00,48.020000,1012.5,21.280000,48.566667,1013.750000,21.090000,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,264.24,-30.60,12.6,99.0,21.7
2,2024-12-10 20:00:00,47.700000,1014.2,20.532500,48.546667,1014.000000,20.396667,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,273.64,-42.09,12.6,100.0,22.0
3,2024-12-10 21:00:00,46.295000,1014.6,19.735000,46.753333,1014.200000,19.586667,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,285.41,-53.48,12.5,100.0,21.7
4,2024-12-10 22:00:00,45.776667,1014.6,19.510000,46.346667,1014.433333,19.196667,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,303.03,-64.02,12.0,100.0,21.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,2025-03-24 18:00:00,47.165000,1011.5,21.363333,46.150000,1010.850000,21.974000,46.7700,1010.900000,21.5475,...,1011.140000,21.5925,3600.0,0.0,0.0,273.44,-1.18,15.5,87.0,21.3
2497,2025-03-24 19:00:00,47.726667,1012.2,21.401429,46.690000,1012.033333,22.076667,47.1720,1011.940000,21.7340,...,1012.200000,21.6880,2366.0,0.0,0.0,283.89,-13.52,14.7,56.0,21.5
2498,2025-03-24 20:00:00,48.800000,1013.2,20.706667,47.763333,1012.900000,21.266667,47.9650,1012.875000,21.0950,...,1013.166667,20.8775,0.0,0.0,0.0,295.64,-25.12,13.5,33.0,21.7
2499,2025-03-24 21:00:00,49.685000,1013.5,20.173333,48.830000,1013.400000,20.486667,48.6850,1013.400000,20.4950,...,1013.650000,20.0950,0.0,0.0,0.0,308.84,-34.89,13.1,41.0,21.8


# Limpieza Oro

In [ ]:
from datetime import time
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from time import sleep

sleep(30)
path_read = 'data/data_silver/final_silver.csv'
df = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])[2:]

# Print de las filas que tienen valores nulos o vacíos en la columna "azimuth_mean"
df[df["azimuth_mean"].isna() | (df["azimuth_mean"] == '')]
df = df.drop(2077).reset_index(drop=True)
df = df.drop(0).reset_index(drop=True)

#-------------------------------------------------#

# Crear columnas con la media entre sensor 1 y sensor 2
df['avg_humidity_1_2'] = df[['sensor.sensor_temperatura_1_humidity', 'sensor.sensor_temperatura_2_humidity']].mean(axis=1)
df['avg_pressure_1_2'] = df[['sensor.sensor_temperatura_1_pressure', 'sensor.sensor_temperatura_2_pressure']].mean(axis=1)
df['avg_temperature_1_2'] = df[['sensor.sensor_temperatura_1_temperature', 'sensor.sensor_temperatura_2_temperature']].mean(axis=1)

# Entrenamos modelos separados para cada variable objetivo si hay datos suficientes
def train_and_predict(feature_col, target_col):
    train_data = df[[feature_col, target_col]].dropna()
    if train_data.empty:
        return None

    X_train = train_data[[feature_col]]
    y_train = train_data[target_col]

    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predecir valores faltantes
    missing_mask = df[target_col].isna() & df[feature_col].notna()
    df.loc[missing_mask, target_col] = model.predict(df.loc[missing_mask, [feature_col]]).round(2)

    return model

# Aplicar modelos
model_humidity = train_and_predict('avg_humidity_1_2', 'sensor.sensor_temperatura_3_humidity')
model_pressure = train_and_predict('avg_pressure_1_2', 'sensor.sensor_temperatura_3_pressure')
model_temperature = train_and_predict('avg_temperature_1_2', 'sensor.sensor_temperatura_3_temperature')

# Verificar cuántos valores se rellenaron
filled_counts = {
    'humidity_filled': df['sensor.sensor_temperatura_3_humidity'].notna().sum(),
    'pressure_filled': df['sensor.sensor_temperatura_3_pressure'].notna().sum(),
    'temperature_filled': df['sensor.sensor_temperatura_3_temperature'].notna().sum()
}

# Mostrar resultados
df_predicted = df[['time', 'sensor.sensor_temperatura_3_humidity',
                   'sensor.sensor_temperatura_3_pressure',
                   'sensor.sensor_temperatura_3_temperature']]

#---------------------------------------------------------#

# Crear columnas con la media entre sensor 1 y sensor 2
df['media_humedad'] = df[['sensor.sensor_temperatura_1_humidity', 'sensor.sensor_temperatura_2_humidity', 'sensor.sensor_temperatura_3_humidity']].mean(axis=1)
df['media_presion'] = df[['sensor.sensor_temperatura_1_pressure', 'sensor.sensor_temperatura_2_pressure','sensor.sensor_temperatura_3_pressure']].mean(axis=1)
df['media_temperatura'] = df[['sensor.sensor_temperatura_1_temperature', 'sensor.sensor_temperatura_2_temperature','sensor.sensor_temperatura_3_temperature']].mean(axis=1)

df = df.drop(columns=[
    'sensor.sensor_temperatura_4_humidity',
    'sensor.sensor_temperatura_4_pressure',
    'sensor.sensor_temperatura_4_temperature',
    'avg_humidity_1_2',
    'avg_pressure_1_2',
    'avg_temperature_1_2'
], errors='ignore')

# Formateamos las columnas predichas a 2 decimales
df[['media_humedad', 'media_presion', 'media_temperatura']] = df[['media_humedad', 'media_presion', 'media_temperatura']].round(2)


df.rename(columns={'elevation_mean': 'elevacion_sol', 'temperature_mean': 'temperatura_exterior','cloud_coverage_mean':'porcentaje_nubes','temperature ℃':'temperatura_calefaccion_y'}, inplace=True)

# Guardar el nuevo archivo
upload_path = "data/data_gold"
guardar_ficheros(owner, repo, upload_path, 'entrenamiento_neuronal_calefaccion_gold.csv', df)
df

✅ entrenamiento_neuronal_calefaccion_gold.csv subido correctamente al repo.


,time,sensor.sensor_temperatura_1_humidity,sensor.sensor_temperatura_1_pressure,sensor.sensor_temperatura_1_temperature,sensor.sensor_temperatura_2_humidity,sensor.sensor_temperatura_2_pressure,sensor.sensor_temperatura_2_temperature,sensor.sensor_temperatura_3_humidity,sensor.sensor_temperatura_3_pressure,sensor.sensor_temperatura_3_temperature,...,suma_ventanas_arriba,suma_ventanas_abajo,azimuth_mean,elevacion_sol,temperatura_exterior,porcentaje_nubes,temperatura_calefaccion_y,media_humedad,media_presion,media_temperatura
0,2024-12-10 21:00:00,46.295000,1014.6,19.735000,46.753333,1014.200000,19.586667,46.0600,1014.370000,19.8100,...,0.0,0.0,285.41,-53.48,12.5,100.0,21.7,46.37,1014.39,19.71
1,2024-12-10 22:00:00,45.776667,1014.6,19.510000,46.346667,1014.433333,19.196667,45.6000,1014.490000,19.5200,...,0.0,0.0,303.03,-64.02,12.0,100.0,21.1,45.91,1014.51,19.41
2,2024-12-10 23:00:00,45.450000,1014.2,19.235000,46.000000,1014.400000,19.050000,45.2600,1014.270000,19.3200,...,0.0,0.0,250.84,-72.16,11.4,100.0,20.8,45.57,1014.29,19.20
3,2024-12-11 00:00:00,45.295000,1014.4,19.090000,45.970000,1014.050000,18.815000,45.1600,1014.200000,19.1500,...,0.0,0.0,29.55,-71.20,11.2,39.0,20.5,45.48,1014.22,19.02
4,2024-12-11 01:00:00,45.250000,1014.4,18.860000,46.335000,1013.950000,18.490000,45.3200,1014.150000,18.8900,...,0.0,0.0,59.16,-63.06,10.6,32.0,20.3,45.64,1014.17,18.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,2025-03-24 18:00:00,47.165000,1011.5,21.363333,46.150000,1010.850000,21.974000,46.7700,1010.900000,21.5475,...,0.0,0.0,273.44,-1.18,15.5,87.0,21.3,46.70,1011.08,21.63
2493,2025-03-24 19:00:00,47.726667,1012.2,21.401429,46.690000,1012.033333,22.076667,47.1720,1011.940000,21.7340,...,0.0,0.0,283.89,-13.52,14.7,56.0,21.5,47.20,1012.06,21.74
2494,2025-03-24 20:00:00,48.800000,1013.2,20.706667,47.763333,1012.900000,21.266667,47.9650,1012.875000,21.0950,...,0.0,0.0,295.64,-25.12,13.5,33.0,21.7,48.18,1012.99,21.02
2495,2025-03-24 21:00:00,49.685000,1013.5,20.173333,48.830000,1013.400000,20.486667,48.6850,1013.400000,20.4950,...,0.0,0.0,308.84,-34.89,13.1,41.0,21.8,49.07,1013.43,20.39


In [ ]:
import numpy as np

path_read = 'data/data_gold/entrenamiento_neuronal_calefaccion_gold.csv'
df = pd.read_csv(f'https://raw.githubusercontent.com/{owner}/{repo}/main/{path_read}', parse_dates=['time'])

df['time'] = pd.to_datetime(df['time'])
df['hora'] = df['time'].dt.hour
df['part_of_day'] = df['hora'].apply(lambda h: (
    0 if h < 6 else #Madrugada = 0
    3 if h < 12 else #Mañana = 3
    2 if h < 18 else  #Tarde = 2
    1 #Noche = 1
))

df['mes'] = df['time'].dt.month
df['season'] = df['hora'].apply(lambda h: (
    0 if h in [12,1,2] else #Invierno = 0
    1 if h in [3,4,5] else #Primavera = 1
    2 if h in [6,7,8] else #Verano = 2
    3 #Otoño = 3
))

df['hora_sin'] = np.sin(2 * np.pi * df['hora'] / 24)
df['hora_cos'] = np.cos(2 * np.pi * df['hora'] / 24)

df["dia_semana"] = df["time"].dt.dayofweek  # Día de la semana (0=lunes)
# Codificación one-hot para dia_semana (ahora solo contiene 0 a 4)
df = pd.get_dummies(df, columns=['dia_semana'], prefix='dia', drop_first=True)

# Eliminar las columnas especificadas
columns_to_drop = [
    "sensor.sensor_temperatura_2_humidity",
    "sensor.sensor_temperatura_2_pressure",
    "sensor.sensor_temperatura_2_temperature",
    "sensor.sensor_temperatura_3_humidity",
    "sensor.sensor_temperatura_3_pressure",
    "sensor.sensor_temperatura_3_temperature",
    "media_humedad", "media_presion", "media_temperatura"
]
df_cleaned = df.drop(columns=columns_to_drop)

# Guardar el nuevo archivo
upload_path = "data/data_gold"
guardar_ficheros(owner, repo, upload_path, 'entrenamiento_neuronal_calefaccion_gold.csv', df_cleaned)

✅ entrenamiento_neuronal_calefaccion_gold.csv subido correctamente al repo.


# Convertir fichero en .ipynb

Lo convierto en este formato para mas adelante poder leerlo sin incriptación desde airflow

In [ ]:
!jupyter nbconvert --to script "Copia de Colab_Sensores.ipynb"

In [ ]:
!cp colab_sensores.py "/content/drive/MyDrive/Colab Notebooks/Programación de inteligencia artificial/Proyecto de sensores/"

In [ ]:
!ls -lh /content